# Pipeline de Processamento de Logs
Este notebook demonstra o uso do pipeline unificado de processamento de logs.

In [1]:
from log_processor import (
    Paths,
    consolidate_logs,
    split_by_gender,
    aggregate_data,
    find_ties,
    remove_ties,
    get_removed_ties
)
import pandas as pd

# Configurando o pandas para mostrar todas as colunas
pd.set_option('display.max_columns', None)

## 1. Consolidação dos Logs
Primeiro, vamos consolidar todos os arquivos de log em um único DataFrame

In [2]:
# Inicializa os caminhos
paths = Paths()

# Consolida os logs
df_consolidated = consolidate_logs(paths)

print(f"Total de registros consolidados: {len(df_consolidated)}")
print("\nPrimeiras linhas dos dados consolidados:")
display(df_consolidated.head())

Total de registros consolidados: 13485

Primeiras linhas dos dados consolidados:


,ParticipantMD5,GeneroCod,frase,Value,duracao
0,bc2584302fffead698e1809107a60040,m,"Já são mais de 5 anos que sofro deste mal, per...",negativa,73.202
1,bc2584302fffead698e1809107a60040,m,Ja estou no limite para chegar a cima do peso.,negativa,2.940
2,bc2584302fffead698e1809107a60040,m,Comprei meu livro.,neutra,1.464
3,bc2584302fffead698e1809107a60040,m,Confesso que aquilo me dava um prazer quase qu...,positiva,5.582
4,bc2584302fffead698e1809107a60040,m,"Mais uma vez eu to aqui me lamentando, eu esto...",negativa,4.070


## 2. Separação por Gênero
Agora vamos separar os dados por gênero e analisar as distribuições

In [3]:
# Separa os dados por gênero
df_male, df_female = split_by_gender(df_consolidated)

# Exibe estatísticas básicas
print(f"Registros masculinos: {len(df_male)}")
print(f"Registros femininos: {len(df_female)}")

# Análise adicional - distribuição de classificações por gênero
print("\nDistribuição de classificações - Masculino:")
display(df_male['Value'].value_counts())

print("\nDistribuição de classificações - Feminino:")
display(df_female['Value'].value_counts())

Registros masculinos: 7025
Registros femininos: 6460

Distribuição de classificações - Masculino:


Value
positiva    2813
negativa    2386
neutra      1826
Name: count, dtype: int64


Distribuição de classificações - Feminino:


Value
neutra      2249
negativa    2162
positiva    2049
Name: count, dtype: int64

## 3. Agregação dos Dados
Vamos agregar os dados por frase e analisar os resultados

In [4]:
# Agrega os dados
male_agg = aggregate_data(df_male)
female_agg = aggregate_data(df_female)

print("Dados agregados - Masculino:")
display(male_agg.head())

print("\nDados agregados - Feminino:")
display(female_agg.head())

# Análise adicional - distribuição das classificações majoritárias
print("\nDistribuição das classificações majoritárias - Masculino:")
display(male_agg['classificacao_majoritaria'].value_counts())

print("\nDistribuição das classificações majoritárias - Feminino:")
display(female_agg['classificacao_majoritaria'].value_counts())

Dados agregados - Masculino:


,frase,duracao_media,total_classificacoes,classificacao_majoritaria,total_positiva,total_negativa,total_neutra
0,"A história fala de três amigas gordinhas, que...",8.46725,4,negativa,1,3,0
1,Ainda mais que tudo que tá acontecendo ultima...,5.56780,5,neutra,1,0,4
2,"Amo você mais do que a mim mesma, afinal você...",2.92420,5,positiva,5,0,0
3,Ela fica jogando na minha cara que eu não sei...,8.11675,4,negativa,0,4,0
4,Ele sempre esteve do meu lado e o que eu mais...,8.63960,5,positiva,3,2,0



Dados agregados - Feminino:


,frase,duracao_media,total_classificacoes,classificacao_majoritaria,total_positiva,total_negativa,total_neutra
0,"A história fala de três amigas gordinhas, que...",15.22775,4,neutra,1,1,2
1,Ainda mais que tudo que tá acontecendo ultima...,9.77775,4,neutra,1,0,3
2,"Amo você mais do que a mim mesma, afinal você...",4.31850,4,empate,2,2,0
3,Ela fica jogando na minha cara que eu não sei...,7.99550,4,negativa,0,4,0
4,Ele sempre esteve do meu lado e o que eu mais...,9.44850,4,empate,2,2,0



Distribuição das classificações majoritárias - Masculino:


classificacao_majoritaria
positiva    561
negativa    477
neutra      284
empate      141
Name: count, dtype: int64


Distribuição das classificações majoritárias - Feminino:


classificacao_majoritaria
negativa    470
positiva    447
neutra      372
empate      174
Name: count, dtype: int64

## 4. Análise de Empates e Remoção
Vamos remover as frases que tiveram empate na classificação majoritária e analisar os resultados


In [5]:
# Remove empates dos dados agregados
male_agg_no_ties = remove_ties(male_agg)
female_agg_no_ties = remove_ties(female_agg)

print("Total de frases após remoção de empates:")
print(f"Masculino: {len(male_agg_no_ties)} (removidas {len(male_agg) - len(male_agg_no_ties)})")
print(f"Feminino: {len(female_agg_no_ties)} (removidas {len(female_agg) - len(female_agg_no_ties)})")

print("\nDistribuição das classificações após remoção de empates - Masculino:")
display(male_agg_no_ties['classificacao_majoritaria'].value_counts())

print("\nDistribuição das classificações após remoção de empates - Feminino:")
display(female_agg_no_ties['classificacao_majoritaria'].value_counts())

Total de frases após remoção de empates:
Masculino: 1322 (removidas 141)
Feminino: 1289 (removidas 174)

Distribuição das classificações após remoção de empates - Masculino:


classificacao_majoritaria
positiva    561
negativa    477
neutra      284
Name: count, dtype: int64


Distribuição das classificações após remoção de empates - Feminino:


classificacao_majoritaria
negativa    470
positiva    447
neutra      372
Name: count, dtype: int64

## 5. Análise das Frases Removidas
Vamos analisar as frases que foram removidas por terem empate

In [6]:
# Obtém informações sobre as frases removidas
removed_ties_df = get_removed_ties(male_agg, female_agg)

print(f"Total de frases removidas por empate: {len(removed_ties_df)}")
print("\nExemplos de frases removidas:")
display(removed_ties_df.head())

# Análise adicional das frases removidas
print("\nDistribuição dos empates:")
empate_stats = {
    'Empate apenas masculino': ((removed_ties_df['empate_masculino'] == 'Sim') & (removed_ties_df['empate_feminino'] == 'Não')).sum(),
    'Empate apenas feminino': ((removed_ties_df['empate_masculino'] == 'Não') & (removed_ties_df['empate_feminino'] == 'Sim')).sum(),
    'Empate em ambos': ((removed_ties_df['empate_masculino'] == 'Sim') & (removed_ties_df['empate_feminino'] == 'Sim')).sum()
}
display(pd.Series(empate_stats))

Total de frases removidas por empate: 289

Exemplos de frases removidas:


,frase,empate_masculino,empate_feminino,total_positiva_m,total_negativa_m,total_neutra_m,total_positiva_f,total_negativa_f,total_neutra_f
0,"Pior que só tem ele lá no horário que vou, e ...",Sim,Não,2,2,1,0,1,3
1,A aula de português estava chata.,Sim,Não,2,0,2,4,0,1
2,"A maturidade bateu em minha porta cedo, eu tin...",Sim,Não,1,2,2,0,4,0
3,A verdade é que sou intensa demais e não há qu...,Sim,Sim,2,0,2,0,2,2
4,"Achei uma solução, ou melhor, na minha cabeça ...",Sim,Sim,2,0,2,2,0,2



Distribuição dos empates:


Empate apenas masculino    115
Empate apenas feminino     148
Empate em ambos             26
dtype: int64

## 6. Salvando os Resultados
Salva todos os DataFrames processados em arquivos CSV

In [7]:
# Salva todos os resultados
df_consolidated.to_csv(paths.consolidated, sep='\t', index=False)
df_male.to_csv(paths.male, sep='\t', index=False)
df_female.to_csv(paths.female, sep='\t', index=False)
male_agg.to_csv(paths.male_agg, sep='\t', index=False)
female_agg.to_csv(paths.female_agg, sep='\t', index=False)
male_agg_no_ties.to_csv(paths.male_agg_no_ties, sep='\t', index=False)
female_agg_no_ties.to_csv(paths.female_agg_no_ties, sep='\t', index=False)
removed_ties_df.to_csv(paths.removed_ties, sep='\t', index=False)

print("Todos os arquivos foram salvos com sucesso!")
for path in [paths.consolidated, paths.male, paths.female, 
            paths.male_agg, paths.female_agg, paths.male_agg_no_ties,
            paths.female_agg_no_ties, paths.removed_ties]:
    print(f"- {path}")

Todos os arquivos foram salvos com sucesso!
- logs_em_tratamento\logs_totais_tratados.csv
- logs_em_tratamento\logs_masculino.csv
- logs_em_tratamento\logs_feminino.csv
- logs_em_tratamento\logs_masculino_agregado.csv
- logs_em_tratamento\logs_feminino_agregado.csv
- logs_em_tratamento\logs_masculino_agregado_sem_empates.csv
- logs_em_tratamento\logs_feminino_agregado_sem_empates.csv
- logs_em_tratamento\frases_removidas_por_empate.csv
